In [1]:
!pip install neo4j

In [2]:
from datetime import datetime

from neo4j import GraphDatabase, basic_auth

In [3]:
URL = "bolt://localhost:7687"
USERNAME = "Daniel_Hurtado"
PASSWORD = "123456"
DATABASE = "taller5"

In [4]:
driver = GraphDatabase.driver(URL, auth = basic_auth(USERNAME, PASSWORD))

In [5]:
db = driver.session(database = DATABASE)

### Creating a basic graph

<img src="imgs/fig1.png">

In [8]:
summary = db.write_transaction(lambda tx: tx.run("CREATE (:Person:Owner {name: 'Pepito'})-[:OWNS]->(:Pet:Cat {name: 'Lolo'})").consume())

In [13]:
summary.counters.properties_set

2

### Attaching a new relationship to an exising node

<img src="imgs/fig2.png">

In [1]:
summary = db.write_transaction(lambda tx: tx.run("MATCH (d:Pet {name: 'Max'}) CREATE (i:Picture {createdAt:  $createdAt}) CREATE (d)-[:APPEARS_IN]->(i)", {"createdAt": datetime.today()}).consume())

NameError: name 'db' is not defined

In [20]:
summary.counters.properties_set

0

### Extending the network

<img src="imgs/fig3.png">

In [21]:
db.write_transaction(lambda tx: tx.run("CREATE (:Person:Owner {name: 'Marcela Reyes'})-[:OWNS]->(:Pet:Cat {name: 'Lukas'})").consume())

In [22]:
db.write_transaction(lambda tx: tx.run("MATCH (i) WHERE ID(i) = 2 MATCH (c:Pet:Cat {name: 'Lukas'}) CREATE (c)-[:APPEARS_IN]->(i)").consume())

In [23]:
db.write_transaction(lambda tx: tx.run("MATCH (c:Pet:Cat {name: 'Lukas'}) CREATE (i:Picture {createdAt:  $createdAt}) CREATE (c)-[:APPEARS_IN]->(i)", {"createdAt": datetime.today()}).consume())

### Counting nodes by type

In [ ]:
result = db.read_transaction(lambda tx: tx.run("MATCH (:Pet) RETURN count(*) AS count").single())

In [ ]:
result["count"]

### Counting number of outbound relationships from a node

In [ ]:
# For a specific node
result = db.read_transaction(lambda tx: tx.run("MATCH (p:Pet) WHERE ID(p) = $node RETURN size((p)-->()) AS count", {"node": 4}).single())

In [ ]:
result["count"]

In [ ]:
# For a group of nodes
result = db.read_transaction(lambda tx: list(tx.run("MATCH (p:Pet)-[r:APPEARS_IN]->(i:Picture) RETURN p.name AS name, COUNT(r) AS num_pictures")))

In [ ]:
for r in result:
    print(r["name"] + " => " + str(r["num_pictures"]))

In [ ]:
db.close()